# Session 1 Part 2: LLM Inference โดยใช้ Unsloth

เนื้อหา:

การฝึกและทำ Inference ของ LLM ที่**ปรับแต่งแล้ว** โดยใช้ Unsloth:
- **FlexAttention**: การทำงาน attention ที่ปรับปรุงโดย Unsloth
- Kernel fusion และการปรับแต่งหน่วยความจำ

โดยจะวัด:
- เวลาในการโหลดโมเดลและหน่วยความจำ
- ความเร็วและ throughput ของ inference
- ความเร็วในการ training (1 epoch บนชุดข้อมูลขนาดเล็ก)
- การใช้หน่วยความจำระหว่างการ training

**โมเดล**: Gemma 2B

---



## Hugging Face Token

ในการเข้าถึงโมเดล Gemma จะต้องยืนยันตัวตนกับ Hugging Face โปรดรับ read access token จาก [Hugging Face](https://huggingface.co/settings/tokens) และเพิ่มลงใน Colab secrets manager ตั้งชื่อ secret ว่า `HF_TOKEN`

หรือ Run Cell ต่อไปนี้เพื่อ Login

In [1]:
from huggingface_hub import notebook_login

notebook_login()

* * *

## เชื่อมต่อกับ T4 GPU

สามารถเชื่อมต่อกับ T4 GPU runtime ไปที่ "**Runtime**" -> "**Change runtime type**" และเลือก "**T4 GPU**" ภายใต้ "**Hardware accelerator**"

ตรวจสอบ GPU

In [2]:
!nvidia-smi

Fri Nov  7 05:17:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

* * *
## การติดตั้ง Library

In [3]:
%%capture
# ติดตั้ง Unsloth พร้อมทุก dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [4]:
import torch
import time
import gc
import numpy as np
import pandas as pd
from datasets import load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer

# ตรวจสอบความพร้อมของ GPU
print(f"CUDA พร้อมใช้งาน: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_props = torch.cuda.get_device_properties(0)
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory(หน่วยความจำ): {gpu_props.total_memory / 1e9:.2f} GB")
    print(f"เวอร์ชัน CUDA: {torch.version.cuda}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
CUDA พร้อมใช้งาน: True
GPU: Tesla T4
GPU Memory(หน่วยความจำ): 15.83 GB
เวอร์ชัน CUDA: 12.6


## การตั้งค่า

In [5]:
# การตั้งค่าโมเดล
MODEL_NAME = "unsloth/gemma-2-2b-it"
MAX_SEQ_LENGTH = 2048

# การตั้งค่าการฝึก - เหมือนกับ BASELINE
NUM_TRAIN_SAMPLES = 500
NUM_TRAIN_EPOCHS = 1
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-4

# prompts สำหรับทดสอบ inference - เหมือนกับ BASELINE
TEST_PROMPTS = [
    "อธิบาย quantum computing ให้เข้าใจง่ายๆ:",
    "ลำดับฟีโบนักชีคืออะไร?",
    "เขียนฟังก์ชัน Python เพื่อกลับข้อความ:"
]

## ฟังก์ชันเสริม

In [6]:
def get_gpu_memory_gb():
    """ดึงข้อมูลการใช้หน่วยความจำ GPU ปัจจุบันเป็น GB"""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1e9
    return 0

def get_peak_memory_gb():
    """ดึงข้อมูลการใช้หน่วยความจำ GPU สูงสุดเป็น GB"""
    if torch.cuda.is_available():
        return torch.cuda.max_memory_allocated() / 1e9
    return 0

def clear_memory():
    """ล้างหน่วยความจำ GPU และระบบ"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

def format_time(seconds):
    """จัดรูปแบบวินาทีให้อ่านได้ง่าย"""
    return f"{seconds:.2f}วินาที ({seconds/60:.2f}นาที)"

def print_section(title):
    """พิมพ์ส่วนหัวที่จัดรูปแบบแล้ว"""
    print(f"\n{'='*80}")
    print(f"{title.center(80)}")
    print(f"{'='*80}\n")

## โหลดชุดข้อมูล

In [7]:
print("กำลังโหลดชุดข้อมูล instruction-tuned...")
dataset = load_dataset("mlabonne/FineTome-100k", split=f"train[:{NUM_TRAIN_SAMPLES}]")

def format_dataset(example):
    """จัดรูปแบบ conversations เป็นคู่ instruction-response"""
    # แยก conversation
    conversations = example.get("conversations", [])
    if len(conversations) < 2:
        return {"text": ""}

    # ดึง instruction ของผู้ใช้และ response ของ assistant
    user_msg = conversations[0].get("value", "")
    assistant_msg = conversations[1].get("value", "")

    # จัดรูปแบบเป็น instruction-response
    text = f"ผู้ใช้: {user_msg}\n\nผู้ช่วย: {assistant_msg}"
    return {"text": text}

dataset = dataset.map(format_dataset, remove_columns=dataset.column_names)
# กรองตัวอย่างที่ว่างออก
dataset = dataset.filter(lambda x: len(x["text"]) > 0)

print(f"โหลดชุดข้อมูลแล้ว: {len(dataset)} ตัวอย่าง")
print(f"ตัวอย่าง:\n{dataset[0]['text'][:200]}...")

กำลังโหลดชุดข้อมูล instruction-tuned...


README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

โหลดชุดข้อมูลแล้ว: 500 ตัวอย่าง
ตัวอย่าง:
ผู้ใช้: Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of...


---

# ขั้นตอนที่ 1: การโหลดโมเดล

โหลด Gemma 2B ด้วย **FastLanguageModel ของ Unsloth**

In [9]:
print_section("กำลังโหลดโมเดลด้วย Unsloth")

clear_memory()
start_time = time.time()
mem_before = get_gpu_memory_gb()

# โหลดโมเดลด้วย FastLanguageModel ของ Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=False,
)

load_time = time.time() - start_time
mem_after = get_gpu_memory_gb()
memory_used = mem_after - mem_before

print(f" โหลดโมเดลเรียบร้อยแล้ว")
print(f"  การทำงานของ Attention: Unsloth FlexAttention")
print(f"  เวลาในการโหลด: {format_time(load_time)}")
print(f"  หน่วยความจำที่ใช้: {memory_used:.3f} GB")
print(f"  หน่วยความจำ GPU ปัจจุบัน: {mem_after:.3f} GB")


                           กำลังโหลดโมเดลด้วย Unsloth                           

==((====))==  Unsloth 2025.11.1: Fast Gemma2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

 โหลดโมเดลเรียบร้อยแล้ว
  การทำงานของ Attention: Unsloth FlexAttention
  เวลาในการโหลด: 133.21วินาที (2.22นาที)
  หน่วยความจำที่ใช้: 5.276 GB
  หน่วยความจำ GPU ปัจจุบัน: 5.276 GB


---

# ขั้นตอนที่ 2: การทดสอบ Inference

ทดสอบความเร็วของ inference ด้วย FlexAttention ของ Unsloth

In [10]:
print_section("การทดสอบ Inference - Unsloth ที่ปรับแต่งแล้ว")

clear_memory()
times = []
outputs = []

# เปิดใช้งานโหมด fast inference
FastLanguageModel.for_inference(model)

inputs = tokenizer(TEST_PROMPTS[0], return_tensors="pt").to(model.device)
with torch.no_grad():
    _ = model.generate(**inputs, max_new_tokens=50, use_cache=True)

clear_memory()
mem_before = get_gpu_memory_gb()

# รัน inference benchmarks
print(f"\nกำลังรัน inference บน {len(TEST_PROMPTS)} prompts...\n")
for i, prompt in enumerate(TEST_PROMPTS, 1):
    print(f"Prompt {i}/{len(TEST_PROMPTS)}: {prompt[:50]}...")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    start = time.time()
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, use_cache=True)
    elapsed = time.time() - start
    times.append(elapsed)

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    outputs.append(decoded)
    print(f"  เวลา: {elapsed:.4f}วินาที")

mem_after = get_gpu_memory_gb()
peak_mem = get_peak_memory_gb()

# ผลลัพธ์
avg_time = np.mean(times)
std_time = np.std(times)
tokens_per_sec = 50 / avg_time

print(f"\n{'='*60}")
print("ผลลัพธ์ INFERENCE")
print(f"{'='*60}")
print(f"เวลาเฉลี่ยต่อ Prompt: {avg_time:.4f}วินาที (±{std_time:.4f}วินาที)")
print(f"Tokens/วินาที: {tokens_per_sec:.2f}")
print(f"หน่วยความจำสูงสุด: {peak_mem:.3f} GB")
print(f"{'='*60}")

# เก็บผลลัพธ์
inference_results = {
    'avg_time': avg_time,
    'std_time': std_time,
    'tokens_per_sec': tokens_per_sec,
    'peak_memory': peak_mem,
    'outputs': outputs
}


                  การทดสอบ Inference - Unsloth ที่ปรับแต่งแล้ว                  


กำลังรัน inference บน 3 prompts...

Prompt 1/3: อธิบาย quantum computing ให้เข้าใจง่ายๆ:...
  เวลา: 2.1109วินาที
Prompt 2/3: ลำดับฟีโบนักชีคืออะไร?...
  เวลา: 2.1193วินาที
Prompt 3/3: เขียนฟังก์ชัน Python เพื่อกลับข้อความ:...
  เวลา: 2.1550วินาที

ผลลัพธ์ INFERENCE
เวลาเฉลี่ยต่อ Prompt: 2.1284วินาที (±0.0191วินาที)
Tokens/วินาที: 23.49
หน่วยความจำสูงสุด: 5.335 GB


## ตัวอย่าง Output

In [11]:
print("\nตัวอย่าง Inference Outputs:\n")
for i, (prompt, output) in enumerate(zip(TEST_PROMPTS, outputs), 1):
    print(f"{'='*80}")
    print(f"Prompt {i}: {prompt}")
    print(f"{'-'*80}")
    print(f"Output: {output[:200]}...")
    print()


ตัวอย่าง Inference Outputs:

Prompt 1: อธิบาย quantum computing ให้เข้าใจง่ายๆ:
--------------------------------------------------------------------------------
Output: อธิบาย quantum computing ให้เข้าใจง่ายๆ:

**คอมพิวเตอร์แบบควอนตัม คือคอมพิวเตอร์ที่ใช้หลักการของควอนตัม (ควอนตัม) ในการทำงาน**

**หลักการของควอนตัม:**

* **...

Prompt 2: ลำดับฟีโบนักชีคืออะไร?
--------------------------------------------------------------------------------
Output: ลำดับฟีโบนักชีคืออะไร?

**ฟีโบนักชี** เป็นลำดับของจำนวนเต็มที่เริ่มต้นด้วย 0 และ 1 และมีการเพิ่มจำนวนขึ้นตามลำดับที่ 
 
 **ลำดับฟีโบนักชี** คือ:...

Prompt 3: เขียนฟังก์ชัน Python เพื่อกลับข้อความ:
--------------------------------------------------------------------------------
Output: เขียนฟังก์ชัน Python เพื่อกลับข้อความ:

**คำอธิบาย:**

*   ฟังก์ชันนี้จะรับข้อความเป็นตัวแปร
*   ฟังก์ชันนี้จะคืนข้อความที่กลับด้าน
*   ฟังก์ชันนี้จะ...



---

# ขั้นตอนที่ 3: การทดสอบการฝึก

Fine-tuning ด้วย LoRA เป็นเวลา 1 epoch เพื่อวัดประสิทธิภาพการฝึก

In [12]:
print_section("การทดสอบการฝึก - Unsloth")

clear_memory()

# เพิ่ม LoRA adapters โดยใช้วิธีของ Unsloth
print("กำลังเพิ่ม LoRA adapters ด้วย Unsloth...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

model.print_trainable_parameters()

# การตั้งค่าการฝึก
training_args = TrainingArguments(
    output_dir="./output_unsloth",
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    save_strategy="no",
    report_to="none",
    optim="adamw_8bit",
)

# สร้าง trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
    dataset_text_field="text",
)

print(f"\nเริ่มการฝึกบน {len(dataset)} ตัวอย่างเป็นเวลา {NUM_TRAIN_EPOCHS} epoch...\n")

# ฝึกและวัด
mem_before = get_gpu_memory_gb()
start_time = time.time()

train_result = trainer.train()

train_time = time.time() - start_time
mem_after = get_gpu_memory_gb()
peak_mem = get_peak_memory_gb()

# ผลลัพธ์
print(f"\n{'='*60}")
print("ผลลัพธ์การฝึก")
print(f"{'='*60}")
print(f"เวลาในการฝึก: {format_time(train_time)}")
print(f"ตัวอย่าง/วินาที: {train_result.metrics['train_samples_per_second']:.2f}")
print(f"หน่วยความจำสูงสุด: {peak_mem:.3f} GB")
print(f"Loss สุดท้าย: {train_result.metrics['train_loss']:.4f}")
print(f"{'='*60}")

# เก็บผลลัพธ์
training_results = {
    'train_time': train_time,
    'samples_per_sec': train_result.metrics['train_samples_per_second'],
    'peak_memory': peak_mem,
    'final_loss': train_result.metrics['train_loss']
}


                            การทดสอบการฝึก - Unsloth                            



Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


กำลังเพิ่ม LoRA adapters ด้วย Unsloth...


Unsloth 2025.11.1 patched 26 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 6,389,760 || all params: 2,620,731,648 || trainable%: 0.2438


...


เริ่มการฝึกบน 500 ตัวอย่างเป็นเวลา 1 epoch...



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 63
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 6,389,760 of 2,620,731,648 (0.24% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.136600
20,1.083900
30,0.936800
40,0.924100
50,0.910800
60,0.930100



ผลลัพธ์การฝึก
เวลาในการฝึก: 350.07วินาที (5.83นาที)
ตัวอย่าง/วินาที: 1.44
หน่วยความจำสูงสุด: 7.565 GB
Loss สุดท้าย: 0.9819


---

# ผลจากการ train ด้วย Unsloth library


In [13]:
print_section("สรุปประสิทธิภาพที่ปรับแต่งแล้วด้วย UNSLOTH")

summary_data = {
    'เมตริก': [
        'เวลาโหลดโมเดล',
        'หน่วยความจำโหลดโมเดล',
        'เวลาเฉลี่ย Inference',
        'Tokens/วินาที Inference',
        'หน่วยความจำสูงสุด Inference',
        'เวลาการฝึก',
        'ตัวอย่าง/วินาที การฝึก',
        'หน่วยความจำสูงสุด การฝึก',
        'Loss สุดท้ายจากการฝึก'
    ],
    'ค่า': [
        format_time(load_time),
        f"{memory_used:.3f} GB",
        f"{inference_results['avg_time']:.4f}วินาที",
        f"{inference_results['tokens_per_sec']:.2f}",
        f"{inference_results['peak_memory']:.3f} GB",
        format_time(training_results['train_time']),
        f"{training_results['samples_per_sec']:.2f}",
        f"{training_results['peak_memory']:.3f} GB",
        f"{training_results['final_loss']:.4f}"
    ]
}

df = pd.DataFrame(summary_data)
print(df.to_string(index=False))

print(f"\n{'='*60}")
print("📝 จุดสำคัญ:")
print(f"{'='*60}")
print("- ใช้ FlexAttention ของ Unsloth (ปรับแต่งแล้ว)")
print("- โมเดล: Gemma 2B (FP16)")
print(f"- การฝึก: {NUM_TRAIN_SAMPLES} ตัวอย่าง, {NUM_TRAIN_EPOCHS} epoch")
print("- LoRA fine-tuning (r=16) ด้วยการปรับแต่งของ Unsloth")
print("- เปิดใช้งาน kernel fusion และการปรับแต่งหน่วยความจำ")
print("\n✅ เปรียบเทียบกับ lab_transformers_baseline.ipynb!")
print(f"{'='*60}")


                   สรุปประสิทธิภาพที่ปรับแต่งแล้วด้วย UNSLOTH                   

                     เมตริก                     ค่า
              เวลาโหลดโมเดล 133.21วินาที (2.22นาที)
       หน่วยความจำโหลดโมเดล                5.276 GB
       เวลาเฉลี่ย Inference            2.1284วินาที
    Tokens/วินาที Inference                   23.49
หน่วยความจำสูงสุด Inference                5.335 GB
                 เวลาการฝึก 350.07วินาที (5.83นาที)
     ตัวอย่าง/วินาที การฝึก                    1.44
   หน่วยความจำสูงสุด การฝึก                7.565 GB
      Loss สุดท้ายจากการฝึก                  0.9819

📝 จุดสำคัญ:
- ใช้ FlexAttention ของ Unsloth (ปรับแต่งแล้ว)
- โมเดล: Gemma 2B (FP16)
- การฝึก: 500 ตัวอย่าง, 1 epoch
- LoRA fine-tuning (r=16) ด้วยการปรับแต่งของ Unsloth
- เปิดใช้งาน kernel fusion และการปรับแต่งหน่วยความจำ

✅ เปรียบเทียบกับ lab_transformers_baseline.ipynb!
